In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2024-10-21 20:21:47.566681: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-21 20:21:48.026544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2024-10-21 20:21:48.778072: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-21 20:21:48.867227: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-21 20:21:48.868784: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.src.legacy.preprocessing.image import ImageDataGenerator
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score

In [3]:
# Mapear arquivos ocultos (.ipny)
def filter_hidden_folders(folder_list):
    return [folder for folder in folder_list if not folder.startswith('.')]

In [4]:
# Caminho para os dados do dataset e o reescalador de imagens

xray_directory = 'font_results/normal'

xray_classes = filter_hidden_folders(os.listdir(xray_directory))  # Filtra pastas ocultas
print(os.listdir(xray_directory))
print(len(os.listdir(xray_directory)))

['g', 'V', 'W', 'r', 'i', 'a', 'G', 'I', 's', 'J', 'y', 'z', 't', 'c', 'Q', 'u', 'E', 'o', 'H', 'B', 'U', 'N', 'F', 'm', 'n', 'v', 'T', 'D', 'A', 'e', 'w', 'O', '.ipynb_checkpoints', 'S', 'K', 'l', 'C', 'q', 'p', 'Z', 'j', 'h', 'f', 'L', 'M', 'b', 'd', 'P', 'x', 'X', 'Y', 'R', 'k']
53


In [5]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Definir parâmetros
input_shape = (180, 100, 1)  # Imagem 180x100, 1 canal (grayscale)
num_classes = 53  # 26 letras maiúsculas, 26 minúsculas, + 1 "null"

# Carregar o modelo VGG16 pré-treinado com pesos do ImageNet
# Incluir a camada de pooling global (retirar as camadas de classificação final)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(180, 100, 3))

# Adaptar a camada de entrada para grayscale (1 canal)
# Expanda o canal para 3 (de 1 para 3) para o modelo pré-treinado que usa RGB
inputs = layers.Input(shape=input_shape)
x = layers.Conv2D(3, (3, 3), padding='same')(inputs)  # Converter 1 canal para 3

# Conectar o modelo pré-treinado
x = base_model(x)
x = layers.GlobalAveragePooling2D()(x)

# Adicionar as camadas de classificação
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(num_classes, activation='softmax')(x)

# Criar o modelo final
model = models.Model(inputs, x)

# Congelar as camadas do modelo pré-treinado para não treiná-las no início
for layer in base_model.layers:
    layer.trainable = False

# Compilar o modelo
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# Resumo do modelo
model.summary()



2024-10-21 20:21:49.222204: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-21 20:21:49.223597: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-21 20:21:49.224903: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 180, 100, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 180, 100, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 5, 3, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 53)             │        27,189 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,004,563 (57.24 MB)

 Trainable params: 289,875 (1.11 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [6]:
# Compilando o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resumo do modelo
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 180, 100, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 180, 100, 3)    │            30 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (None, 5, 3, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 53)             │        27,189 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,004,563 (57.24 MB)

 Trainable params: 289,875 (1.11 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [7]:
# Salvar o arquivo de treinamento em diferentes epocas

# Nome/caminho do arquivo que sera salvo
filepath = "weights_normal_vgg16.keras" 
#filepath = "weights_normal_vgg16.keras" 

# Define os parametros para atualizar o arquivo final
# ModelCheckpoint(nome_do_arquivo, monitor = 'parametro a ser alalisado', verbose = 1 (imprimir no console quando arquivo for salvo), save_best_only = True, mode = 'min' (objetivo minimizar perda))
checkpoint = ModelCheckpoint(filepath, monitor = 'loss', verbose = 1, save_best_only = True, mode = 'min')

# Cria uma lista que contém o ModelCheckpoint configurado usado durante o treinamento
callbacks_list = [checkpoint]

In [8]:
# Gerenciador de imagem para pre processamento
# Aumentar a base de dados 

train_datagen = ImageDataGenerator(
# Transforma os valores de pixel de (0-255 ->0.0 - 1.0 )  
    rescale=1. / 255,
# Aplica zoom aleatório às imagens dentro do intervalo especificado (0.2 -> 20%)
    zoom_range=0.2,
#Este parâmetro é usado para dividir automaticamente os dados em conjuntos de treinamento e validação 
    validation_split=0.3
    )


In [9]:
# Cria critério de parada precoce
early_stopping = EarlyStopping(monitor='loss', patience=3, verbose=1)

In [10]:

BATCH_SIZE = 50
TARGET_SIZE=(180, 100)
CMODE='grayscale'
CLASS_MODE='categorical'
# Imagens para treinamento
train_generator = train_datagen.flow_from_directory(
    xray_directory,
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    color_mode=CMODE, 
    class_mode=CLASS_MODE,
    shuffle=True,
    subset='training'
)

# Imagens para validação
validation_generator = train_datagen.flow_from_directory(
        xray_directory,
        target_size=TARGET_SIZE,
        batch_size=BATCH_SIZE,
        color_mode=CMODE,
        class_mode=CLASS_MODE,
        shuffle=True,
        subset = 'validation'
        )
print(train_generator.class_indices)



Found 2658448 images belonging to 53 classes.
Found 1139268 images belonging to 53 classes.
{'.ipynb_checkpoints': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}


In [11]:
# Testar o gerador de dados
for data_batch, labels_batch in train_generator:
    print("Imagens: ", data_batch.shape)
    print("Labels: ", labels_batch.shape)
    break  # Apenas testar um batch


Imagens:  (50, 180, 100, 1)
Labels:  (50, 53)


steps_per_epoch = len(train_generator) - 1  # Total de batches por época

# treinamento do modelo 


In [ ]:
history = model.fit(train_generator, epochs = 20, validation_data=validation_generator, callbacks=callbacks_list)
aux = history
history = history.history.keys() #paramestros de avaliação accuracy, loss

Epoch 1/25


/home/picg/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1729552946.450685   18073 service.cc:145] XLA service 0x73463400d890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1729552946.450703   18073 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4070, Compute Capability 8.9
2024-10-21 20:22:26.472858: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-21 20:22:26.579666: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1729

    3/53169 ━━━━━━━━━━━━━━━━━━━━ 56:30 64ms/step - accuracy: 0.0400 - loss: 4.0376

I0000 00:00:1729552954.008287   18073 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 1399/53169 ━━━━━━━━━━━━━━━━━━━━ 52:25 61ms/step - accuracy: 0.0911 - loss: 3.6019

I0000 00:00:1729553039.477547   18497 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1084', 436 bytes spill stores, 436 bytes spill loads

I0000 00:00:1729553039.501893   18513 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_868', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1729553039.705302   18494 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1084', 428 bytes spill stores, 428 bytes spill loads

I0000 00:00:1729553039.920410   18497 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'triton_gemm_dot_1075', 212 bytes spill stores, 212 bytes spill loads



53169/53169 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3516 - loss: 2.3538
Epoch 1: loss improved from inf to 1.99205, saving model to weights_normal_vgg16.keras
53169/53169 ━━━━━━━━━━━━━━━━━━━━ 3850s 72ms/step - accuracy: 0.3516 - loss: 2.3538 - val_accuracy: 0.3855 - val_loss: 2.3516
Epoch 2/25
53169/53169 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5350 - loss: 1.5979
Epoch 2: loss improved from 1.99205 to 1.55530, saving model to weights_normal_vgg16.keras
53169/53169 ━━━━━━━━━━━━━━━━━━━━ 3832s 72ms/step - accuracy: 0.5350 - loss: 1.5979 - val_accuracy: 0.3963 - val_loss: 2.4371
Epoch 3/25
53169/53169 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5699 - loss: 1.4621
Epoch 3: loss improved from 1.55530 to 1.44675, saving model to weights_normal_vgg16.keras
53169/53169 ━━━━━━━━━━━━━━━━━━━━ 3831s 72ms/step - accuracy: 0.5699 - loss: 1.4621 - val_accuracy: 0.4210 - val_loss: 2.2447
Epoch 4/25
53169/53169 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.5856 - loss: 1.4017
Epo

In [ ]:
# Exemplo de grafico para ver taxa de acerto por taxa de erro 
plt.plot(aux.history['accuracy'])
plt.plot(aux.history['loss'])
plt.title('Erro e taxa de acerto durante o treinamento')
plt.xlabel('Época')
plt.ylabel('Taxa de acerto e erro')
plt.legend(['Taxa de acerto', 'Erro']);

In [ ]:
hist_df = pd.DataFrame(aux.history)
print(hist_df)

In [ ]:
# Salvar o DataFrame em um arquivo CSV
hist_df.to_csv('training_history_normal_vgg16.csv', index=False)

In [ ]:
# Carregar os dados do CSV
loaded_history_df = pd.read_csv('training_history_normal_vgg16.csv')
# Exibir os dados carregados (opcional)
print(loaded_history_df)

In [ ]:
# Plotar a perda de treinamento e validação
plt.plot(loaded_history_df['loss'], label='Loss (train)')
plt.plot(loaded_history_df['val_loss'], label='Loss (validation)')
plt.title('Loss durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.show()

# Plotar a precisão de treinamento e validação
plt.plot(loaded_history_df['accuracy'], label='Accuracy (train)')
plt.plot(loaded_history_df['val_accuracy'], label='Accuracy (validation)')
plt.title('Precisão durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()
plt.show()

In [ ]:
#Comparação de LOSS e ACCURACY entre as epocas de treino da rede

# Plot Loss
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(loaded_history_df['loss'], label='train loss')
plt.plot(loaded_history_df['val_loss'], label='val loss')
plt.title('Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Plot Accuracy
plt.subplot(1, 2, 2)
plt.plot(loaded_history_df['accuracy'], label='train accuracy')
plt.plot(loaded_history_df['val_accuracy'], label='val accuracy')
plt.title('Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()


In [ ]:
#Comparação de Loss e Accuracy de redes distintas

import numpy as np

def plot_comparison(results, metric='accuracy'):
    models = list(results.keys())
    final_accuracy = [results[model][metric][-1] for model in models]
    
    plt.figure(figsize=(10, 5))
    plt.bar(models, final_accuracy)
    plt.title(f'Comparação de {metric} entre diferentes redes')
    plt.ylabel(metric.capitalize())
    plt.xlabel('Modelos')
    plt.show()


In [ ]:
#Grafico de curvas para comparação entre redes
def plot_comparison_curves(results, metric='accuracy'):
    plt.figure(figsize=(10, 5))
    
    for model, history in results.items():
        plt.plot(history[metric], label=f'{model} - train')
        plt.plot(history['val_' + metric], '--', label=f'{model} - val')
    
    plt.title(f'Comparação de {metric.capitalize()} entre Redes')
    plt.xlabel('Épocas')
    plt.ylabel(metric.capitalize())
    plt.legend()
    plt.show()


In [ ]:
#Matriz Confusão
from sklearn.metrics import confusion_matrix
import seaborn as sns

def plot_confusion_matrix(y_true, y_pred, model_name):
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusão - {model_name}')
    plt.ylabel('Verdadeiro')
    plt.xlabel('Predito')
    plt.show()



In [ ]:
#Boxplot
def plot_boxplot(results, metric='accuracy'):
    data = [history[metric] for history in results.values()]
    labels = results.keys()

    plt.figure(figsize=(10, 6))
    plt.boxplot(data, labels=labels)
    plt.title(f'Boxplot de {metric.capitalize()} entre Redes')
    plt.ylabel(metric.capitalize())
    plt.show()


In [ ]:
# defini caminho para pasta de teste
test_directory = 'dataset/test'
# lista contendo os nomes dos arquivos e diretórios presentes no caminho fornecido
os.listdir(test_directory)

In [ ]:
# Mudar a escala das imagens
test_gen = ImageDataGenerator(rescale=1./255)

test_generator = test_gen.flow_from_directory(batch_size = 13, directory = test_directory,
                                              shuffle = True, target_size = (256, 256),
                                              class_mode = 'categorical')

evaluate = model.evaluate(test_generator)

len(os.listdir(test_directory))

In [ ]:
# aplica a rede treinada em um conjunto de teste de imagens e armazena os resultados
prediction = []
original = []
image = []

# percorre as subpastas no diretório test_directory, assumindo que cada subpasta contém imagens de uma determinada classe
for i in range(len(os.listdir(test_directory))):
  # percorre os arquivos dentro de cada subpasta e processa cada imagem
  for item in os.listdir(os.path.join(test_directory, str(i))):
    
    #Etapas de processamento de img

    img = cv2.imread(os.path.join(test_directory, str(i), item))
    img = cv2.resize(img, (256, 256))
    image.append(img)
    img = img / 255
    img = img.reshape(-1, 256, 256, 3)

    #rede neural aplicada
    predict = model.predict(img)
    predict = np.argmax(predict)
    prediction.append(predict)
    original.append(i)

In [ ]:
accuracy_score(original, prediction)

In [ ]:
fig, axes = plt.subplots(5, 5, figsize=(12,12))
axes = axes.ravel()
for i in np.arange(0, 25):
  axes[i].imshow(image[i])
  axes[i].set_title('Previsão={}\nTrue={}'.format(str(labels_names[prediction[i]]), str(labels_names[original[i]])))
  axes[i].axis('off')
plt.subplots_adjust(wspace = 1.2)

In [ ]:
labels_names = {0: 'SoldasCorretas', 1: 'SoldasErradas'}

In [ ]:
from keras.models import load_model
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Carregar o modelo previamente salvo
model_load = load_model('weights.keras')

# Carregar a imagem e convertê-la para um array numpy em escala de cinza
image = np.array(Image.open("/home/picg/TCC/GeradorBase/font_results/normal/a/a-mas-GrandHotel-Regular.png").convert('L'))

# Redimensionar a imagem para o tamanho esperado pela CNN (supondo 180x100)
temp_resized = cv2.resize(image, (180, 100))

print(temp_resized.shape)

# Normalizar os valores dos pixels para [0, 1]
normalized = temp_resized / 255.0

# Adicionar uma dimensão para corresponder ao formato de entrada esperado pelo modelo
final = normalized.reshape(1, 180, 100, 1)  # '1' aqui é o batch size

# Fazer a predição com o modelo carregado
prediction = model_load.predict(final)

# Exibir o resultado da predição
print("Predição:", prediction)


In [ ]:
plt.imshow(temp_resized)

In [ ]:
#image = np.array(Image.open("/home/picg/TCC/GeradorBase/font_results/normal/a/a-mam-GrandHotel-Regular.png"))
plt.imshow(image, cmap="gray")

In [ ]:
import numpy as np

# Achar a classe com maior probabilidade
predicted_class = np.argmax(prediction)
'''
{'.ipynb_checkpoints': 0, 'A': 1, 'B': 2, 'C': 3, 'D': 4, 'E': 5, 'F': 6, 'G': 7, 'H': 8, 'I': 9, 'J': 10, 'K': 11, 'L': 12, 'M': 13, 'N': 14, 'O': 15, 'P': 16, 'Q': 17, 'R': 18, 'S': 19, 'T': 20, 'U': 21, 'V': 22, 'W': 23, 'X': 24, 'Y': 25, 'Z': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52}
'''

# Definir labels (saidas) 
labels_names = {0: '.ipynb_checkpoints', 
                1: 'a',
                2: 'A',
                3: 'b',
                4: 'B',
                5: 'c',
                6: 'C',
                7: 'd',
                8: 'D',
                9: 'e',
                10: 'E', 
                11: 'f',
                12: 'F',
                13: 'g',
                14: 'G',
                15: 'h',
                16: 'H',
                17: 'i',
                18: 'I',
                19: 'j',
                20: 'J', 
                21: 'k',
                22: 'K',
                23: 'l',
                24: 'L',
                25: 'm',
                26: 'M',
                27: 'n',
                28: 'N',
                29: 'o',
                30: 'O', 
                31: 'p',
                32: 'P',
                33: 'q',
                34: 'Q',
                35: 'r',
                36: 'R',
                37: 's',
                38: 'S',
                39: 't',
                40: 'T', 
                41: 'u',
                42: 'U',
                43: 'v',
                44: 'V',
                45: 'w',
                46: 'W',
                47: 'x',
                48: 'X',
                49: 'y',
                50: 'Y', 
                51: 'z',
                52: 'Z',
                }

print(f"A classe prevista é - {predicted_class}: {labels_names[predicted_class]}")


In [ ]:
model_load.summary()